In [7]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
   print("Starting authentication process...")
   try:
       # Load credentials from the service account file
       credentials = service_account.Credentials.from_service_account_file(
           'credentials.json', 
           scopes=SCOPES
       )
       print("Successfully loaded service account credentials")
       
       # Build the service
       service = build('sheets', 'v4', credentials=credentials)
       print("Successfully created Google Sheets service")
       return service
       
   except Exception as e:
       print(f"Error during authentication: {str(e)}")
       raise
   


# 🌐 Web Scraping Functions
    It has inbuit scraper to scrape the website content and store it in the google sheet
    
    NOTE:

    1. Google sheets Should have a column named "organization_website_url" with the website urls to scrape
    2. Google sheets Should have a column named "website_content" to store the scraped website content in a single column if not present it will be created.
    3. It only scrapes the first 5 subpages of the website and concatenates the content of the subpages to the main page content
    4. It also limits the content to 100 words to avoid Wasteing Openai API credits


In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse 
import urljoin
import time
from urllib.parse import urlparse, urljoin

from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
   print("Starting authentication process...")
   try:
       # Load credentials from the service account file
       credentials = service_account.Credentials.from_service_account_file(
           'credentials.json', 
           scopes=SCOPES
       )
       print("Successfully loaded service account credentials")
       
       # Build the service
       service = build('sheets', 'v4', credentials=credentials)
       print("Successfully created Google Sheets service")
       return service
       
   except Exception as e:
       print(f"Error during authentication: {str(e)}")
       raise

def get_subpages(base_url):
   """Get list of subpages from the main URL"""
   try:
       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
       response = requests.get(base_url, headers=headers, timeout=10)
       response.raise_for_status()
       
       soup = BeautifulSoup(response.text, 'html.parser')
       base_domain = urlparse(base_url).netloc
       
       subpages = set()
       for link in soup.find_all('a', href=True):
           url = urljoin(base_url, link['href'])
           # Only include URLs from same domain and avoid parameters
           if urlparse(url).netloc == base_domain and '#' not in url and '?' not in url:
               subpages.add(url)
               
       return list(subpages)
   except Exception as e:
       print(f"Error getting subpages for {base_url}: {str(e)}")
       return []
   
def get_text_from_url(url):
   """Extract clean text content from a URL"""
   try:
       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
       response = requests.get(url, headers=headers, timeout=10)
       response.raise_for_status()
       
       soup = BeautifulSoup(response.text, 'html.parser')
       
       # Remove unwanted elements
       for element in soup(['script', 'style', 'nav', 'footer', 'header']):
           element.decompose()
           
       # Get text and clean it
       text = ' '.join(soup.stripped_strings)
       return text.strip()
   except Exception as e:
       print(f"Error scraping {url}: {str(e)}")
       return ""

def scrape_website_content(service, spreadsheet_id):
    try:
        # Get the existing data
        range_name = 'Sheet1'
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        rows = result.get('values', [])
        
        if not rows:
            print('No data found.')
            return
        
        # Identify or create the website_content column
        header = rows[0]
        if 'website_content' not in header:
            header.append('website_content')
            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f'{range_name}!A1',
                valueInputOption='RAW',
                body={'values': [header]}
            ).execute()
        
        content_col_idx = header.index('website_content')  # Get column index
        content_col_name = get_column_letter(content_col_idx + 1)  # Convert to column name
        
        # Process each row
        for i, row in enumerate(rows[1:], start=2):  # Start at row 2 in the sheet
            if len(row) > header.index('organization_website_url'):  # Check if URL exists
                url = row[header.index('organization_website_url')]
                content_exists = (
                    len(row) > content_col_idx and row[content_col_idx].strip() != ""
                )
                if url and not content_exists:
                    print(f"Processing row {i}: {url}")
                    if not url.startswith("http://") and not url.startswith("https://"):
                        url = f"http://{url}"
      # Default to HTTP if no protocol is provided
                    
                    # Scrape website content
                    content = get_text_from_url(url)
                    words = content.split()
                    
                    # If content is less than 100 words, try subpages
                    if len(words) < 100:
                        subpages = get_subpages(url)
                        for subpage in subpages[:5]:  # Limit to first 5 subpages
                            sub_content = get_text_from_url(subpage)
                            content += " " + sub_content
                            words = content.split()
                            if len(words) >= 100:
                                break
                            time.sleep(0.3)  # Be nice to the server
                    
                    # Limit content to 100 words
                    trimmed_content = " ".join(words[:100])
                    
                    # Update the cell for website content
                    cell = f"{content_col_name}{i}"
                    service.spreadsheets().values().update(
                        spreadsheetId=spreadsheet_id,
                        range=f'Sheet1!{cell}',
                        valueInputOption='RAW',
                        body={'values': [[trimmed_content]]}
                    ).execute()
                    
                    print(f"Row {i} updated in column {content_col_name}")
                    time.sleep(0.2)  # Be nice to servers
                else:
                    print(f"Skipping row {i}: Content already exists or no URL.")
        
        print("Website content scraping completed successfully!")
        
    except Exception as e:
        print(f"Error: {str(e)}")


def get_column_letter(col_idx):
    """Convert a 1-based column index to a column letter (e.g., 1 -> A, 27 -> AA)."""
    col_letter = ""
    while col_idx > 0:
        col_idx, remainder = divmod(col_idx - 1, 26)
        col_letter = chr(65 + remainder) + col_letter
    return col_letter



def main():
   service = get_google_sheets_service()
   spreadsheet_id = '1Xr13JkH27PWV-EKrhh65JtAqGAhtUg1PHJ78Qy4Y8uw'
   scrape_website_content(service, spreadsheet_id)
   
#    service = get_subpages('https://www.aircraftinteriorsexpo.com/')
#    for subpage in service:
#        content = get_text_from_url(subpage)
#        print("subpage\n", content )
#    print("website content:\n",service)



if __name__ == '__main__':
   main()

# ✉️ Email Verification System
    # Run this cell below to verify the emails in the sheet.
    
    NOTE: Google sheets Should have
    1. column named "email" with emails to verify 
    2. column named "verified_email" to store the verified emails

In [ ]:


import requests
from google.oauth2 import service_account
from googleapiclient.discovery import build
import time

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
   print("Starting authentication process...")
   try:
       # Load credentials from the service account file
       credentials = service_account.Credentials.from_service_account_file(
           'credentials.json', 
           scopes=SCOPES
       )
       print("Successfully loaded service account credentials")
       
       # Build the service
       service = build('sheets', 'v4', credentials=credentials)
       print("Successfully created Google Sheets service")
       return service
       
   except Exception as e:
       print(f"Error during authentication: {str(e)}")
       raise
   
def get_column_letter(col_idx):
    """Convert a 1-based column index to a column letter (e.g., 1 -> A, 27 -> AA)."""
    col_letter = ""
    while col_idx > 0:
        col_idx, remainder = divmod(col_idx - 1, 26)
        col_letter = chr(65 + remainder) + col_letter
    return col_letter

def verify_email(email):
    """Verify email using Bouncify API"""
    try:
        api_key = ""
        url = f"https://api.bouncify.io/v1/verify?apikey={api_key}&email={email}"
        
        response = requests.get(url)
        response.raise_for_status()
        
        result = response.json()
        print("APIresponse:",result)
        # Check if the API call was successful
        if not result.get("success"):
            return "Error: " + result.get("result", "Unknown error")
            
        # Get the verification result
        status = result.get("result")
        
        # Map the API response to verification status
        if status == "deliverable":
            return "Valid"
        elif status == "undeliverable":
            return "Invalid"
        elif status in ["accept-all", "unknown"]:
            return "Invalid"  # These could be valid addresses
        else:
            return f"Unknown: {status}"
            
    except requests.exceptions.RequestException as e:
        if e.response is not None:
            if e.response.status_code == 401:
                return "Error: Invalid API Key"
            elif e.response.status_code == 402:
                return "Error: Insufficient credits"
            elif e.response.status_code == 429:
                time.sleep(5)  # Wait 5 seconds if rate limited
                return "Error: Rate limited"
        return f"Error: {str(e)}"



def verify_emails_in_sheet(service, spreadsheet_id):
    try:
        # Get the existing data
        range_name = 'Sheet1'
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        rows = result.get('values', [])
        
        if not rows:
            print('No data found.')
            return
            
        # Find or create the email verification column
        header = rows[0]
        if 'verified_email' not in header:
            header.append('verified_email')
            verified_col_idx = len(header) - 1
        else:
            verified_col_idx = header.index('verified_email')
            
        email_col_idx = header.index('email')  # Get email column index
        verified_col_name = get_column_letter(verified_col_idx + 1)
        
        # Process each row
        for i, row in enumerate(rows[1:], start=2):
            if len(row) > email_col_idx:
                email = row[email_col_idx]
                if email:
                    # First check if the cell already has a verified email
                    cell = f"{verified_col_name}{i}"
                    cell_result = service.spreadsheets().values().get(
                        spreadsheetId=spreadsheet_id,
                        range=f'Sheet1!{cell}'
                    ).execute()
                    
                    # Check if cell is empty
                    cell_empty = not cell_result.get('values', [])
                    
                    if cell_empty:
                        print(f"Processing row {i}: {email}")
                        verification_status = verify_email(email)
                        
                        # Update the cell with the email only if status is Valid
                        if verification_status == "Valid":
                            service.spreadsheets().values().update(
                                spreadsheetId=spreadsheet_id,
                                range=f'Sheet1!{cell}',
                                valueInputOption='RAW',
                                body={'values': [[email]]}
                            ).execute()
                            print(f"Added verified email: {email}")
                        else:
                            # Clear the cell if email is invalid
                            service.spreadsheets().values().update(
                                spreadsheetId=spreadsheet_id,
                                range=f'Sheet1!{cell}',
                                valueInputOption='RAW',
                                body={'values': [["-"]]}
                            ).execute()
                            print(f"Skipped invalid email: {email} (Status: {verification_status})")
                    else:
                        print(f"Skipping row {i}: Email already verified")
                    time.sleep(0.2)  # Rate limiting

                    
                    
        print("Email verification completed successfully!")
        
    except Exception as e:
        print(f"Error: {str(e)}")

# Update the main function to include email verification
def main():
    print("##############################################")
    # verify_email("james.stackhouse@stackedfitness.com.au")
    # verify_email("jim.kokkinakis@theeyepractice.com.au")
    # verify_email("tanusri@herro.com")
    # verify_email("tanusri@fernzz.com")
    print("##############################################")
    service = get_google_sheets_service()
    spreadsheet_id = '1Xr13JkH27PWV-EKrhh65JtAqGAhtUg1PHJ78Qy4Y8uw'
    verify_emails_in_sheet(service, spreadsheet_id)

if __name__ == '__main__':
    main()

# 🧪Lead Enrichment

### 1. Researching About Prospects company
### 2. Crafting personalized offers 

In [ ]:
import json

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
   print("Starting authentication process...")
   try:
       # Load credentials from the service account file
       credentials = service_account.Credentials.from_service_account_file(
           'credentials.json', 
           scopes=SCOPES
       )
       print("Successfully loaded service account credentials")
       
       # Build the service
       service = build('sheets', 'v4', credentials=credentials)
       print("Successfully created Google Sheets service")
       return service
       
   except Exception as e:
       print(f"Error during authentication: {str(e)}")
       raise
   
   
def get_column_letter(col_idx):
    """Convert a 1-based column index to a column letter (e.g., 1 -> A, 27 -> AA)."""
    col_letter = ""
    while col_idx > 0:
        col_idx, remainder = divmod(col_idx - 1, 26)
        col_letter = chr(65 + remainder) + col_letter
    return col_letter

def analyze_with_gpt4(content):
    """Helper function to make API calls to Azure's GPT-4o-mini"""
    API_KEY = ""
    ENDPOINT = ""
    
    payload = {
        "messages": [
            {"role": "system", "content": "You are a lead qualification expert. Analyze the provided content and return a JSON response."},
            {"role": "user", "content": f"""
                Analyze this website content and provide the following information in JSON format:
                Website content: {content[:1000]}
                
                Please classify this lead with the following details:
                1. lead_category: (Hot, Warm, Cold)
                2. industry: (specific industry)
                3. company_size: (Small, Medium, Large)
                4. lead_score: (1-100)
                
                Base your classification on factors like:
                - Company size and scope
                - Industry relevance
                - Professional language
                - Business maturity
                
                Respond ONLY with a JSON object containing these four fields.
            """}
        ],
        "temperature": 0.7,
        "max_tokens": 100,
        "response_format": { "type": "json_object" }
    }
    
    headers = {
        "api-key": API_KEY,
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(
            ENDPOINT,
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        print("response:",response)
        result = response.json()
        print("result:",result['choices'][0]['message']['content'])
        return result['choices'][0]['message']['content']

    except Exception as e:
        print(f"API call error: {str(e)}")
        raise

def enrich_leads(service, spreadsheet_id):
    """Enrich leads using Azure's GPT-4o-mini to analyze website content"""
    try:
        # Get the existing data
        range_name = 'Sheet1'
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        rows = result.get('values', [])
        
        if not rows:
            print('No data found.')
            return
            
        # Find or create necessary columns
        header = rows[0]
        new_columns = ['lead_category', 'industry', 'company_size', 'lead_score']
        
        # Add new columns if they don't exist
        for column in new_columns:
            if column not in header:
                header.append(column)
                
        # Get column indices
        website_content_idx = header.index('website_content')
        category_idx = header.index('lead_category')
        industry_idx = header.index('industry')
        size_idx = header.index('company_size')
        score_idx = header.index('lead_score')
        
        # Process each row
        for i, row in enumerate(rows[1:], start=2):
            if len(row) > website_content_idx and row[website_content_idx].strip():
                website_content = row[website_content_idx]
                
                # Skip if already processed
                if len(row) > category_idx and row[category_idx].strip():
                    print(f"Skipping row {i}: Already enriched")
                    continue
                
                print(f"Processing row {i}")
                
                try:
                    # Get analysis from GPT-4o-mini
                    analysis_response = analyze_with_gpt4(website_content)
                    analysis = json.loads(analysis_response)
                    
                    # Update the sheet with enriched data
                    updates = [
                        [analysis['lead_category']],
                        [analysis['industry']],
                        [analysis['company_size']],
                        [str(analysis['lead_score'])]
                    ]
                    
                    # Update each column
                    for col_idx, value in zip(
                        [category_idx, industry_idx, size_idx, score_idx],
                        updates
                    ):
                        col_letter = get_column_letter(col_idx + 1)
                        service.spreadsheets().values().update(
                            spreadsheetId=spreadsheet_id,
                            range=f'Sheet1!{col_letter}{i}',
                            valueInputOption='RAW',
                            body={'values': value}
                        ).execute()
                    
                    print(f"Successfully enriched row {i}")
                    time.sleep(1)  # Rate limiting
                    
                except json.JSONDecodeError:
                    print(f"Error parsing API response for row {i}")
                    continue
                except Exception as e:
                    print(f"Error processing row {i}: {str(e)}")
                    continue
                
    except Exception as e:
        print(f"Error during lead enrichment: {str(e)}")

# Update the main function
def main():
    analyze_with_gpt4("hello")
    service = get_google_sheets_service()
    spreadsheet_id = '1Xr13JkH27PWV-EKrhh65JtAqGAhtUg1PHJ78Qy4Y8uw'
    enrich_leads(service, spreadsheet_id)

if __name__ == '__main__':
    main()

In [10]:
import json
import requests
from google.oauth2 import service_account
from googleapiclient.discovery import build
import time

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
   print("Starting authentication process...")
   try:
       # Load credentials from the service account file
       credentials = service_account.Credentials.from_service_account_file(
           'credentials.json', 
           scopes=SCOPES
       )
       print("Successfully loaded service account credentials")
       
       # Build the service
       service = build('sheets', 'v4', credentials=credentials)
       print("Successfully created Google Sheets service")
       return service
       
   except Exception as e:
       print(f"Error during authentication: {str(e)}")
       raise
   
   
def get_column_letter(col_idx):
    """Convert a 1-based column index to a column letter (e.g., 1 -> A, 27 -> AA)."""
    col_letter = ""
    while col_idx > 0:
        col_idx, remainder = divmod(col_idx - 1, 26)
        col_letter = chr(65 + remainder) + col_letter
    return col_letter

def analyze_website_content(content):
    """Helper function to generate website summaries using GPT-4o-mini"""
    API_KEY = ""
    ENDPOINT = ""
    
    payload = {
        "messages": [
            {"role": "system", "content": "You are an expert at summarizing website content concisely and professionally."},
            {"role": "user", "content": f"""
                Analyze this website content and provide a JSON response with the following:
                Website content: {content[:1000]}
                
                Create a summary with these components:
                1. brief_summary: (2-3 sentence overview)
                2. key_offerings: (list of main products/services)
                3. target_audience: (who the website targets)
                4. unique_value_proposition: (what makes them stand out)
                
                Respond ONLY with a JSON object containing these four fields.
            """}
        ],
        "temperature": 0.7,
        "max_tokens": 150,
        "response_format": { "type": "json_object" }
    }
    
    headers = {
        "api-key": API_KEY,
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(
            ENDPOINT,
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        result = response.json()

        print(result['choices'][0]['message']['content'])
        return result['choices'][0]['message']['content']
    except Exception as e:
        print(f"API call error: {str(e)}")
        raise
def generate_website_summaries(service, spreadsheet_id):
    """Generate summaries for website content using Azure's GPT-4o-mini"""
    try:
        # Get the existing data
        range_name = 'Sheet1'
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        rows = result.get('values', [])
        
        if not rows:
            print('No data found.')
            return
            
        # Find or create necessary columns
        header = rows[0]
        new_columns = ['brief_summary', 'key_offerings', 'target_audience', 'unique_value_proposition']
        columns_added = False
        
        # Check which columns need to be added
        columns_to_add = [col for col in new_columns if col not in header]
        if columns_to_add:
            # Add new columns to header
            header.extend(columns_to_add)
            
            # Update the header row in the sheet
            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f'{range_name}!A1',
                valueInputOption='RAW',
                body={'values': [header]}
            ).execute()
            print(f"Added new columns: {columns_to_add}")
            columns_added = True
        
        # If columns were added, get fresh data with new structure
        if columns_added:
            result = service.spreadsheets().values().get(
                spreadsheetId=spreadsheet_id,
                range=range_name
            ).execute()
            rows = result.get('values', [])
            header = rows[0]
                
        # Get column indices
        website_content_idx = header.index('website_content')
        summary_idx = header.index('brief_summary')
        offerings_idx = header.index('key_offerings')
        audience_idx = header.index('target_audience')
        uvp_idx = header.index('unique_value_proposition')
        
        # Process each row
        for i, row in enumerate(rows[1:], start=2):
            if len(row) > website_content_idx and row[website_content_idx].strip():
                website_content = row[website_content_idx]
                
                # Skip if already processed
                if len(row) > summary_idx and row[summary_idx].strip():
                    print(f"Skipping row {i}: Already summarized")
                    continue
                
                print(f"Processing row {i}")
                
                try:
                    # Get analysis from GPT-4o-mini
                    analysis_response = analyze_website_content(website_content)
                    analysis = json.loads(analysis_response)
                    
                    # Process each field separately
                    for col_idx, field in zip(
                        [summary_idx, offerings_idx, audience_idx, uvp_idx],
                        ['brief_summary', 'key_offerings', 'target_audience', 'unique_value_proposition']
                    ):
                        col_letter = get_column_letter(col_idx + 1)
                        value = str(analysis.get(field, ''))  # Convert to string and handle missing fields
                        
                        # Update single cell with proper formatting
                        range_name = f'Sheet1!{col_letter}{i}'
                        body = {
                            'values': [[value]]  # Double nested array for single cell
                        }
                        
                        service.spreadsheets().values().update(
                            spreadsheetId=spreadsheet_id,
                            range=range_name,
                            valueInputOption='RAW',
                            body=body
                        ).execute()
                    
                    print(f"Successfully summarized row {i}")
                    time.sleep(1)  # Rate limiting
                    
                except json.JSONDecodeError as e:
                    print(f"Error parsing API response for row {i}: {e}")
                    continue
                except Exception as e:
                    print(f"Error processing row {i}: {str(e)}")
                    continue
                
    except Exception as e:
        print(f"Error during website summarization: {str(e)}")

# Update the main function
def main():
    service = get_google_sheets_service()
    spreadsheet_id = '1Xr13JkH27PWV-EKrhh65JtAqGAhtUg1PHJ78Qy4Y8uw'
    generate_website_summaries(service, spreadsheet_id)

if __name__ == '__main__':
    main()

Starting authentication process...
Successfully loaded service account credentials
Successfully created Google Sheets service
Added new columns: ['brief_summary', 'key_offerings', 'target_audience', 'unique_value_proposition']
Skipping row 2: Already summarized
Processing row 3
{
  "brief_summary": "Optimal Media specializes in scaling home service contractors through targeted Google and Facebook advertising. They focus on eliminating inefficiencies to achieve faster results for their clients.",
  "key_offerings": [
    "Google Ads",
    "Facebook Ads",
    "Free Consultation",
    "Free Audit Call"
  ],
  "target_audience": "Local service businesses looking to increase sales and customers.",
  "unique_value_proposition": "Their approach avoids formalities and inefficiencies, allowing them to deliver effective advertising solutions tailored specifically for the local service industry."
}
Successfully summarized row 3
Processing row 4
{
  "brief_summary": "Step Digital positions itself 

KeyboardInterrupt: 